# FreshHarvest Model Explainability

This notebook provides explainability analysis for FreshHarvest models using various interpretation techniques.

## Explainability Overview
- Grad-CAM visualizations
- Feature importance analysis
- Layer activation visualizations
- Model decision interpretation
- Prediction confidence analysis

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Import custom modules
from cvProject_FreshHarvest.utils.common import read_yaml, setup_logging

# Setup
setup_logging()
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("FreshHarvest Model Explainability Notebook")
print("=" * 50)
print(f"TensorFlow version: {tf.__version__}")

## 1. Model and Configuration Loading

In [ ]:
# Load configuration
config = read_yaml('../config/config.yaml')
CLASS_NAMES = [
    'F_Banana', 'F_Lemon', 'F_Lulo', 'F_Mango', 'F_Orange', 'F_Strawberry', 'F_Tamarillo', 'F_Tomato',
    'S_Banana', 'S_Lemon', 'S_Lulo', 'S_Mango', 'S_Orange', 'S_Strawberry', 'S_Tamarillo', 'S_Tomato'
]

# Load trained model
model_paths = [
    '../models/trained/best_model.h5',
    '../models/checkpoints/best_model_20250618_100126.h5'
]

model = None
for path in model_paths:
    if os.path.exists(path):
        try:
            model = keras.models.load_model(path)
            print(f"✅ Model loaded from: {path}")
            break
        except Exception as e:
            print(f"❌ Failed to load {path}: {e}")

if model is None:
    print("⚠️ No model found. Creating dummy model for demonstration.")
    model = keras.Sequential([
        keras.layers.Input(shape=(224, 224, 3)),
        keras.layers.Conv2D(32, 3, activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(16, activation='softmax')
    ])

## 2. Grad-CAM Implementation

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generate Grad-CAM heatmap."""
    
    # Create gradient model
    grad_model = keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    # Compute gradients
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Generate heatmap
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

def find_last_conv_layer(model):
    """Find the last convolutional layer."""
    for layer in reversed(model.layers):
        if 'conv' in layer.name.lower():
            return layer.name
    return None

# Find last conv layer
last_conv_layer = find_last_conv_layer(model)
print(f"Last conv layer: {last_conv_layer}")

## 3. Sample Image Analysis

In [ ]:
# Load sample images for analysis
def load_sample_image(image_path, target_size=(224, 224)):
    """Load and preprocess a sample image."""
    try:
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array, img
    except Exception as e:
        print(f"Error loading image: {e}")
        # Create dummy image
        img_array = np.random.random((1, 224, 224, 3))
        img = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)
        return img_array, img

# Try to find sample images
sample_paths = []
data_dirs = ['../data/processed/test', '../data/processed/val']

for data_dir in data_dirs:
    if os.path.exists(data_dir):
        for class_name in CLASS_NAMES[:4]:  # First 4 classes
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                if images:
                    sample_paths.append((os.path.join(class_dir, images[0]), class_name))
                    break
        if sample_paths:
            break

if not sample_paths:
    print("⚠️ No sample images found. Using dummy data.")
    sample_paths = [('dummy.jpg', 'F_Banana')]

print(f"Found {len(sample_paths)} sample images for analysis")

## 4. Grad-CAM Visualization

In [ ]:
# Generate Grad-CAM visualizations
def visualize_gradcam(img_path, class_name, model, last_conv_layer):
    """Create Grad-CAM visualization."""
    
    # Load image
    img_array, original_img = load_sample_image(img_path)
    
    # Make prediction
    preds = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(preds[0])
    confidence = preds[0][predicted_class]
    
    print(f"\nAnalyzing: {class_name}")
    print(f"Predicted: {CLASS_NAMES[predicted_class]} (confidence: {confidence:.3f})")
    
    if last_conv_layer:
        # Generate Grad-CAM
        try:
            heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer, predicted_class)
            
            # Create visualization
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            # Original image
            if isinstance(original_img, np.ndarray):
                axes[0].imshow(original_img)
            else:
                axes[0].imshow(original_img)
            axes[0].set_title(f'Original\n{class_name}')
            axes[0].axis('off')
            
            # Heatmap
            axes[1].imshow(heatmap, cmap='jet')
            axes[1].set_title('Grad-CAM Heatmap')
            axes[1].axis('off')
            
            # Overlay
            if isinstance(original_img, np.ndarray):
                overlay_img = original_img
            else:
                overlay_img = np.array(original_img)
            
            heatmap_resized = cv2.resize(heatmap, (224, 224))
            heatmap_colored = plt.cm.jet(heatmap_resized)[:, :, :3]
            overlay = 0.6 * overlay_img/255.0 + 0.4 * heatmap_colored
            
            axes[2].imshow(overlay)
            axes[2].set_title(f'Overlay\nPred: {CLASS_NAMES[predicted_class][:8]}... ({confidence:.3f})')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"Error generating Grad-CAM: {e}")
    else:
        print("No convolutional layer found for Grad-CAM")

# Generate visualizations for sample images
for img_path, class_name in sample_paths[:3]:  # Analyze first 3 samples
    visualize_gradcam(img_path, class_name, model, last_conv_layer)